In [ ]:
import requests
import json
import time

# get api key from Leap AI https://www.tryleap.ai
API_KEY = "YOUR API KEY"

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {API_KEY}"
}

In [ ]:
def generate_image(model_id, prompt):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences"

    payload = {
        "prompt": prompt,
        "steps": 50,
        "width": 512,
        "height": 512,
        "numberOfImages": 4,
        "promptStrength": 7,
        "enhancePrompt": False,
        "restoreFaces": True,
        "upscaleBy": "x2"
    }

    response = requests.post(url, json=payload, headers=HEADERS)
    data = json.loads(response.text)

    print(data)
    inference_id = data["id"]
    status = data["status"]

    print(f"Inference ID: {inference_id}. Status: {status}")

    return inference_id, status


def get_inference_job(model_id, inference_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences/{inference_id}"

    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    state = data["state"]
    images = None

    if len(data["images"]):
        images = data["images"]

    print(f"Inference ID: {inference_id}. State: {state}")

    return inference_id, state, images




In [ ]:
# modelID refers to Realistic Vision model. We can now use it to generate images from. 
# For more models go here: https://docs.tryleap.ai/reference/pre-trained-models
model_id = "eab32df0-de26-4b83-a908-a83f3015e971"
inference_id, status = generate_image(
    model_id, 
    prompt="close up of face of beautiful woman fashion model in yellow feather clothes, official vogue magazine photoshoot, dramatic lighting highly detailed, with the city of paris in the background"
)
while status != "finished":
    time.sleep(10)
    inference_id, status, images = get_inference_job(model_id, inference_id)

for image in images:
    print("Image ready:", image["uri"])